In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import random
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import re
from pymouse import PyMouse
import sys

# 正在出售房屋資料爬蟲
url : https://www.realtor.com/realestateandhomes-search/98010/type-single-family-home/pg-1

In [2]:
# 獲得房屋網址
urls = [] 
for zipcode in ['98010', '98029', '98092', '98075', 98106]:
    for i in range(1, 3):
        url = 'https://www.realtor.com/realestateandhomes-search/%s/type-single-family-home/pg-%s' % (zipcode , str(i))
        headers = {'user-agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
        cookies = {'cookie': 'threshold_value=40; automation=false; clstr=; clstr_tcv=; split=n; split_tcv=194; __vst=eddd0c98-d93a-4277-9b4c-16f2a9154a52; __ssn=cc1241c2-0a13-4652-b517-4a6d2a05dadf; __ssnstarttime=1625931673; __edwssnstarttime=1625931673; __split=80; bcc=false; bcvariation=SRPBCRR%3Av1%3Adesktop; header_slugs=; criteria=loc%3D110th+Ave+SE%2C+98006%26locSlug%3D98006%2F110th-Ave-SE%26status%3D8%26pg%3D1%26pgsz%3D44%26sprefix%3Dpropertyrecord-search; _rdc-next_session=SEZzUVpveSs0cDZxY3FBdUZzb1BmNkNCWkhhanNnTUlKck5sUkpnckxTclc4Q2d2RnhNMFZyaU55ZnBpVmtyUlMxY3ZJMDA2RmJsWWh3UGV4SUxOYnR4U1dsd1dnWEE5TWppM3dHUVdzQjhSTlo4YW0zZHZzcmpPcUwyZ3JQbEdxSkRRWTB5SHo3bzJ5d0dxb0lkb2Vzbml1bGdXU1EwV1JnWi80by8wMDJzU3R3VlBOZlM2VnJBTFBiZTFvbmJZcDU0Q1ZNTDF2UFRCQnZlMnJ6c0krRUxpZkhMZDFtb05KSXpYdXUwRDA2bjRqbndXbnZ4aTF6aWkvejRLdXhBcS0tZ08xRnY3czFQUzVHMDRHTGc3TVl4QT09--f37738dae63a1961861fdac9f85db985d2a84e24; AMCVS_8853394255142B6A0A4C98A4%40AdobeOrg=1; s_ecid=MCMID%7C02366094880149816592795432562425202192; userStatus=new_user; pxcts=45823530-e195-11eb-a378-8b717057deba; _pxvid=4581f7f5-e195-11eb-be0c-0242ac120019; _pxff_bdd=2000; _pxff_cde=5,10; ajs_anonymous_id=%22ec20c98f-f1e6-4a13-aaf3-0151435691c1%22; AMCV_8853394255142B6A0A4C98A4%40AdobeOrg=-1124106680%7CMCIDTS%7C18819%7CMCMID%7C02366094880149816592795432562425202192%7CMCAAMLH-1626536479%7C11%7CMCAAMB-1626536479%7C6G1ynYcLPuiQxYZrsz_pkqfLG9yMXBpb2zX5dvJdYQJzPXImdj0y%7CMCOPTOUT-1625938880s%7CNONE%7CMCAID%7CNONE%7CMCSYNCSOP%7C411-18826%7CvVersion%7C5.2.0; QSI_HistorySession=https%3A%2F%2Fwww.realtor.com%2F%2Fpropertyrecord-search%2F98006%2F110th-Ave-SE~1625931681460; _ga=GA1.2.423422079.1625931682; _gid=GA1.2.145667913.1625931682; _gat=1; _gcl_au=1.1.1385353512.1625931682; AMCVS_AMCV_8853394255142B6A0A4C98A4%40AdobeOrg=1; AMCV_AMCV_8853394255142B6A0A4C98A4%40AdobeOrg=-1124106680%7CMCMID%7C02366094880149816592795432562425202192%7CMCIDTS%7C18819%7CMCOPTOUT-1625938882s%7CNONE%7CvVersion%7C5.2.0; adcloud={%22_les_v%22:%22y%2Crealtor.com%2C1625933482%22}; _fbp=fb.1.1625931682580.1029353258; _uetsid=47413950e19511ebb7b1bfab6737e05c; _uetvid=47441f10e19511eb9846a38020b5fc67; _ncg_sp_ses.cc72=*; _ncg_id_=781921f9-0621-42bc-8196-badef060a6db; _ncg_sp_id.cc72=781921f9-0621-42bc-8196-badef060a6db.1625931683.1.1625931684.1625931683.5e572dde-fff4-43ae-8280-e74e8c6a3f95; __qca=P0-1194499900-1625931683803; _px3=0e89711af4bc87178605b27fb31ff7303f0a37fed79f556a42bc8b830c53490c:sFVezVwXvrz8jHRJmmh9pdMTzWoCd0/85cIXPv9KVvXc13Z1zYzr89sjdIaxdZvdWVvIlf7mroweCKo/RgWKVg==:1000:3kbiuQ1Eg8D/JSmNt3L0vJ4WvverwItKaelZz0BZXjlxkOkTlDMvq2UEIk4Y7K7MSj+ODx7RT/txPmeBWroaEVKl22Gu5yeL/aNyzE8D9WaGHzkA0gOfCn3Ml0EwkjRiDM0iNKC93CM7Tq3ciGdElj8bibdmfARuJURFLKQ+w1Xlml0y2ihsi8Htm6aNMwryqcKK3B1/uu2g5R/xRPGXWA=='} 

        res = requests.get(url, headers=headers, cookies=cookies)
        bs = BeautifulSoup(res.text, 'html.parser')
        for i in bs.find('ul', attrs={'data-testid' : "property-list-container"}).find_all('li', attrs={"data-testid" : "result-card"}):
            url = i.find('div', attrs={'data-testid' : "pc-photo-wrap"}).find('a')['href']
            urls.append('https://www.realtor.com/' + url)
        time.sleep(random.randint(100,150))

In [3]:
len(urls)

321

In [4]:
# 點掉反爬蟲機器人
def check_root(): 
    time.sleep(random.randint(2,4))
    m = PyMouse()
    x= random.randint(360,540)
    y = random.randint(540,590)
    mx = random.randint(0,10)
    my = random.randint(0,10)
    m.press(x,y)
    time.sleep(random.randint(3,5))
    m.release(x+mx, y+my)
    print('check_robot')

In [5]:
# 爬取房間數、廁所數、房屋面積、土地面積
def bed_bath_sqft_lot(bs):
    div = bs.find('div' ,id = 'ldp-property-meta').find('ul')
    bed = div.find('li', attrs={'data-label':"property-meta-beds"})
    bath = div.find('li', attrs={'data-label':"property-meta-bath"})
    sqft = div.find('li', attrs={'data-label':"property-meta-sqft"})
    lot = div.find('li', attrs={'data-label':"property-meta-lotsize"})
    if bed:
        bed = bed.find('span').text
    if bath:
        bath = bath.find('span').text
    if sqft:
        sqft = sqft.text.replace(',','').replace('\n','').replace(' ','')
    if lot:
        lot = lot.text.replace(',','').replace('\n','').replace(' ','')
    return bed, bath, sqft, lot

In [6]:
# 爬取地址
def address(bs):
    address = bs.find('div', id='ldp-address')['content']
    return address


In [7]:
# 爬取國小、國中、高中學校分數
def school(bs):
    try:
        school = []
        table = bs.find(id='load-more-schools').find('table').find_all('tr')[1:]
        for i in table[:3]:
            score = i.find_all('td')[0].text.replace('\n','')
            school.append(score)
        return school[0], school[1], school[2]
    except AttributeError:
        return None, None, None

In [8]:
# 爬取房屋型態、建造年份
def property_year(bs):
    houseyear = bs.find('div', id='key-fact-carousel').find('ul').find('li', attrs={'data-label':"property-year"}).find('div', class_="key-fact-data ellipsis").text
    propertyType = bs.find('div', id='key-fact-carousel').find('ul').find('li', attrs={'data-label':"property-type"}).find('div', class_="key-fact-data ellipsis").text
    return houseyear, propertyType

In [9]:
# 爬取該房房屋稅
def pt_ta(bs):
    try:
        table = bs.find(id='ldp-history-taxes').find('table').find_all('tr')[2]
        year = table.find_all('td')[0].text
        tax = table.find_all('td')[1].text.replace('$','').replace(',','')
        totalAssessed = table.find_all('td')[6].text.replace('$','').replace(',','')
        return tax, totalAssessed
    except AttributeError:
        return None, None

In [10]:
# 爬取賣家出價及日期
def price_date(bs):
    try:
        price, date = None, None
        table = bs.find(id='ldp-history-price').find('table').find('tbody').find('tr')
        date = table.find('td').text
        price = table.find_all('td')[2].text.replace('$','').replace(',','')
        return price, date
    except AttributeError:
        return None, None

In [16]:
newTable = []
errorUrl = []
error = 0
chrom_opt = webdriver.ChromeOptions()
prefs = { "profile.managed_default_content_settings.images": 2 }
chrom_opt.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome('./chromedriver', chrome_options=chrom_opt)
for url in urls[:]:
    try:
        info3 = []
        driver.set_window_size(800, 1000)
        driver.implicitly_wait(5)
        driver.get(url)
        time.sleep((random.randint(3,5)))
        bs = BeautifulSoup(driver.page_source, 'html.parser')
        
        if bs.find('div', id='ldp-address') == None:
            count = 0
            while bs.find('title').text == 'Pardon Our Interruption':
                if count == 6:
                    print('robot error')
                    time.sleep(20)
                    break
                
                time.sleep(1)
                ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform() 
                check_root() 
                time.sleep(3)     
                count += 1
                bs = BeautifulSoup(driver.page_source, 'html.parser')    
        
        bedBathSqft = bed_bath_sqft_lot(bs)     
        bed = bedBathSqft[0]
        bath = bedBathSqft[1]
        sqft = bedBathSqft[2]
        lot = bedBathSqft[3]
        
        add = address(bs)
        
        schoolScore = school(bs)
        school1 = schoolScore[0]
        school2 = schoolScore[1]
        school3 = schoolScore[2]
        
        typeYearLot = property_year(bs)
        yearBulit = typeYearLot[0]
        PropertyType = typeYearLot[1]
        
        ptta = pt_ta(bs)
        pt = ptta[0]
        ta = ptta[1]
        
        priceAndDate = price_date(bs)
        price = priceAndDate[0]
        date =priceAndDate[1]
        
        newTable.append([bed, bath, sqft, date, add, school1, school2, school3, PropertyType, yearBulit, lot, pt, ta, price])
        time.sleep(random.randint(3,5))
# 發生錯誤時儲存報錯網址即報錯原因 方便知道問題在哪   
    except:
        error += 1
        s=sys.exc_info()
        print(url)
        errorText = "Error '%s' happened on line %d" % (s[1],s[2].tb_lineno)
        errorUrl.append([url, errorText])
        print(errorText)
        if 'chrome not reachable' in errorText or 'HTTPConnectionPool' in errorText:
            print('internet error')
            break
            
        time.sleep(random.randint(3,5))

# 資料清洗

In [15]:
df = pd.DataFrame(newTable, columns=['bed', 'bath', 'sqft', 'date', 'address', 'school1', 'school2', 'school3',
                                     'PropertyType', 'yearBulit', 'lot', 'pt', 'ta', 'price'])

In [17]:
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)

In [19]:
df['bed'] = df['bed'].str.replace('+','')
df['bath'] = df['bath'].str.replace('+','')
df['sqft'] = df['sqft'].str.replace('sqft','')
df['school1'] = df['school1'].str.replace('NR', '0')
df['school2'] = df['school2'].str.replace('NR', '0')
df['school3'] = df['school3'].str.replace('NR', '0')

In [20]:
for i in range(len(df)):
    lot = df.iloc[i, 10]
    if 'sqft' in lot:
        df.iloc[i, 10] = int(df.iloc[i, 10].replace('sqftlot', ''))
    if 'acres' in lot:
        df.iloc[i,10] = int(float(df.iloc[i, 10].replace('acreslot', '')) * 43560)

In [21]:
df.iloc[:, 0:2] = df.iloc[:, 0:2].astype(float)
df.iloc[:, 2] = df.iloc[:, 2].astype(int)
df.iloc[:, 5:8] = df.iloc[:, 5:8].astype(int)
df.iloc[:, 9:] = df.iloc[:, 9:].astype(int)

In [22]:
df['yearBulit'] = df['date'].str.replace('[0-9]{1,2}/[0-9]{1,2}/',  '').astype(int) - df['yearBulit']

In [23]:
df.head()

,bed,bath,sqft,date,address,school1,school2,school3,PropertyType,yearBulit,lot,pt,ta,price
0,3.0,1.0,1040,08/05/2021,"21729 Se 288th St, Black Diamond, WA 98010",7,3,4,Single Family Home,47,14810,3237,260000,369888
1,1.0,1.0,400,06/11/2021,"31019 229th Ave Se, Black Diamond, WA 98010",7,3,4,Single Family Home,41,17424,2047,156000,250000
2,3.0,1.0,2983,06/12/2021,"29630 232nd Ave Se, Black Diamond, WA 98010",9,0,7,Single Family Home,40,51836,5150,463000,899000
3,4.0,1.0,1540,08/06/2021,"32330 2nd Ave, Black Diamond, WA 98010",4,5,5,Single Family Home,29,5940,3136,285000,499950
4,3.0,2.0,2020,07/16/2021,"29620 232nd Ave Se, Black Diamond, WA 98010",9,0,7,Single Family Home,32,61419,4974,446000,735000


In [24]:
df.to_csv('DemoData.csv', index=0)